# 📊 8.3 — Laboratorio: Dashboard Interactivo de Ventas

En este laboratorio construiremos un **dashboard analítico de ventas** utilizando **Plotly Express** y **ipywidgets**.

💡 *Objetivo:* combinar los conocimientos de Pandas y visualización para crear un panel interactivo que permita explorar las ventas por fecha, producto y región (simulada).

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

print('✅ Notebook 8.3 — Dashboard Interactivo cargado correctamente.')

---
## 1️⃣ Cargar y preparar datos

Usaremos el dataset `ventas.csv` y añadiremos una columna `region` para poder segmentar las visualizaciones.

In [ ]:
df = pd.read_csv('../datasets/ventas.csv')
df.rename(columns={'cantidad': 'unidades', 'precio': 'precio_unitario'}, inplace=True)
df['fecha'] = pd.to_datetime(df['fecha'])
df['ingresos'] = df['unidades'] * df['precio_unitario']

# Asignar regiones simuladas
regiones = ['Norte', 'Sur', 'Este', 'Oeste']
df['region'] = np.random.choice(regiones, len(df))

df.head()

---
## 2️⃣ Crear controles interactivos (filtros)

Usaremos **ipywidgets** para permitir la selección de producto y rango de fechas.

In [ ]:
# Selector de producto
productos = df['producto'].unique().tolist()
selector_producto = widgets.SelectMultiple(
    options=productos,
    value=[productos[0]],
    description='Producto:',
    style={'description_width': 'initial'}
)

# Selector de rango de fechas
rango_fechas = widgets.SelectionRangeSlider(
    options=sorted(df['fecha'].dt.strftime('%Y-%m-%d').unique()),
    index=(0, len(df['fecha'].unique()) - 1),
    description='Rango de fechas:',
    layout={'width': '600px'}
)

display(selector_producto, rango_fechas)

---
## 3️⃣ Función de actualización del dashboard

La función `actualizar_dashboard` filtrará los datos y generará 3 gráficos:

- 📈 Evolución temporal de ingresos
- 🧭 Ingresos por región
- 🍰 Distribución por producto

In [ ]:
def actualizar_dashboard(productos, rango):
    fecha_inicio, fecha_fin = pd.to_datetime(rango)
    df_filtrado = df[
        (df['producto'].isin(productos)) &
        (df['fecha'] >= fecha_inicio) & (df['fecha'] <= fecha_fin)
    ]
    
    # Gráfico 1 — Ingresos por fecha
    ingresos_dia = df_filtrado.groupby('fecha', as_index=False)['ingresos'].sum()
    fig1 = px.line(ingresos_dia, x='fecha', y='ingresos', title='Evolución de Ingresos', markers=True)
    fig1.update_layout(template='plotly_white')
    
    # Gráfico 2 — Ingresos por región
    ingresos_region = df_filtrado.groupby('region', as_index=False)['ingresos'].sum()
    fig2 = px.bar(ingresos_region, x='region', y='ingresos', title='Ingresos por Región', color='region',
                  color_discrete_sequence=px.colors.qualitative.Vivid)
    fig2.update_layout(template='plotly_white')
    
    # Gráfico 3 — Distribución por producto
    ingresos_producto = df_filtrado.groupby('producto', as_index=False)['ingresos'].sum()
    fig3 = px.pie(ingresos_producto, values='ingresos', names='producto',
                  title='Participación por Producto', color_discrete_sequence=px.colors.qualitative.Pastel)
    fig3.update_traces(textinfo='percent+label')
    
    fig1.show()
    fig2.show()
    fig3.show()

---
## 4️⃣ Conectar widgets con el dashboard

Usaremos `widgets.interactive` para vincular la función de actualización con los filtros definidos.

In [ ]:
widgets.interactive(
    actualizar_dashboard,
    productos=selector_producto,
    rango=rango_fechas
)

---
## 5️⃣ 🧩 Ejercicio — Métricas adicionales

Añade debajo del dashboard un **resumen numérico** con:

- Total de ingresos
- Promedio de unidades vendidas
- Producto más vendido (por ingresos)

👉 *Tip:* puedes usar `df_filtrado.agg()` y `idxmax()`.

In [ ]:
# ✏️ Tu código aquí...

### ✅ Solución propuesta

In [ ]:
def resumen(df_filtrado):
    total_ingresos = df_filtrado['ingresos'].sum()
    promedio_unidades = df_filtrado['unidades'].mean().round(2)
    producto_top = df_filtrado.groupby('producto')['ingresos'].sum().idxmax()
    print(f"💰 Total ingresos: {total_ingresos:.2f} €")
    print(f"📦 Promedio de unidades: {promedio_unidades}")
    print(f"🏆 Producto más vendido: {producto_top}")

---
## 6️⃣ Exportar Dashboard

Puedes exportar cualquiera de los gráficos en formato HTML interactivo o imagen.

In [ ]:
# fig1.write_html('dashboard_ingresos.html')
# print('✅ Dashboard exportado como dashboard_ingresos.html')

---
## 7️⃣ 🧠 Conclusiones

- Los widgets permiten interactividad directa con los datos.
- Plotly facilita la creación de visualizaciones profesionales.
- Este enfoque es ideal para construir dashboards ligeros en notebooks sin necesidad de frameworks adicionales.